In [64]:
import torch
from IPython import display
from d2l import torch as d2l

In [65]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [66]:
num_inputs = 784
num_outputs = 10
W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)
W.shape[0]
b.shape

torch.Size([10])

In [67]:
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True)
    return X_exp / partition #这里运用了广播机制

In [68]:
X = torch.normal(0, 1, (2,5))
X_prob = softmax(X)
X_prob,X_prob.sum(1, keepdim=True)

(tensor([[0.1811, 0.2175, 0.1502, 0.1226, 0.3285],
         [0.4460, 0.2711, 0.1540, 0.1010, 0.0279]]),
 tensor([[1.],
         [1.]]))

In [69]:
#实现softmax回归模型
def net(x):
    return softmax(torch.matmul(X.reshape((-1, X.shape[0])), w)+b)

创建一个数据y_hat,其中包含2个样本在3个类别的预测概率，使用y作为y_hat中概率的索引

In [70]:
#对待第0个样本，看它预测为第0类别的概率；对待第1个样本，看它预测为第2个类别的概率
y = torch.tensor([0,2])
y_hat = torch.tensor([[0.1, 0.3, 0.6],[0.3, 0.2, 0.5]])
y_hat[[0,1],y]#意思是指在y_hat[0,0]、y_hat[1,2]处取值

tensor([0.1000, 0.5000])

In [71]:
#定义交叉熵损失函数
def cross_entropy(y_hat, y):
    '''
    即索引y轴，在x轴上根据labels抽取预测值
    '''
    return -torch.log(y_hat[range(len(y_hat)), y])
cross_entropy(y_hat, y)

tensor([2.3026, 0.6931])

In [72]:
#将预测类别与真实y元素进行比较
def accuracy(y_hat, y):
    '''计算预测正确的数量'''
    if len(y_hat.shape)>1 and y_hat.shape[1]>1:
        y_hat = y_hat.argmax(axis=1)#axis=1，沿着行方向行进，找出在每一行中最大概率对应的下标
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

accuracy(y_hat,y) /len(y)

0.5

In [73]:
#可以评估在任意模型net的准确率
def evaluate_accuracy(net, data_iter):
    if isinstance(net, torch.nn.Module):
        net.eval()
    metric = Accumulator(2)
    for X,y in data_iter:
        metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]